<a href="https://colab.research.google.com/github/njainds/Colab_notebooks/blob/master/CCP_ICD10_Searchengine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# requirements
import nltk
import numpy as np
import operator
import requests
import flask
from keras.preprocessing.text import Tokenizer
import re
import string
from wordcloud import STOPWORDS
! pip install pyspellchecker
from spellchecker import SpellChecker
from gensim.models.keyedvectors import KeyedVectors


    100% |████████████████████████████████| 1.9MB 17.4MB/s 


In [0]:
# Initializers
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/CCP-ICDsearch/wikipedia-pubmed-and-PMC-w2v.bin', binary=True)
wpt = nltk.WordPunctTokenizer()
spell = SpellChecker()

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CCP-ICDsearch"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
CD.npy	ICD_dict.npy  map_abbreviation.npy  wikipedia-pubmed-and-PMC-w2v.bin


In [0]:
# Functions
# Building vectors for CD
prohibitedWords = ['other','age-related','unspecified','[any part, except buttock]','[any part, except face]','not elsewhere classified','part','parts','specific','region','site','sites','specified','Unspecified','Generalized','(s)','multiple','due to medical condition','type','initial encounter','subsequent encounter']
big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))

def onetimeclean(text):
  x = wpt.tokenize(big_regex.sub("", text.lower()))
  x = [w for w in x if w not in list(STOPWORDS)+['(',')','[',']','-',"'",':',',','.','{','}','_']]
  vect = np.array([np.zeros((200,)) if word not in model.wv.vocab else model.wv[word] for word in x])
  if vect.shape[0]<11:
    vect = np.vstack((vect,np.zeros((11-vect.shape[0],200))))
  if vect.shape != (11,200): print("vector shape not consistent: {}".format(text))
  return vect

CD = np.array([onetimeclean(code) for code in dict(ICD_dict).values()])
np.save('/content/drive/My Drive/CCP-ICDsearch/CD.npy', CD)

def clean(text):
  x = wpt.tokenize(text.lower())
  x = [w for w in x if w not in list(STOPWORDS)+['(',')','-',"'",':',',','.','{','}','_']]
  x = [map_abbreviation[word] if word in list(map_abbreviation.keys()) else word for word in x]
  x = list(set([spell.correction(word) if word not in model.wv.vocab else word for word in x ]))
  vect = np.array([np.zeros((200,)) if word not in model.wv.vocab else model.wv[word] for word in x])
  return vect

In [0]:
# KB
ICD_dict = np.load('/content/drive/My Drive/CCP-ICDsearch/ICD_dict.npy')
ICD_dict = sorted(ICD_dict..items(), key=lambda kv: kv[0])
map_abbreviation = np.load('/content/drive/My Drive/CCP-ICDsearch/map_abbreviation.npy').item() 
CD = np.load('/content/drive/My Drive/CCP-ICDsearch/CD.npy')


In [0]:
DD = clean(search_term)
out = operator.itemgetter(*list(np.mean(np.dot(CD,DD.T),axis=(1,2)).argsort()[-5:][::-1]))(ICD_dict)
print(out)

In [0]:
search_term = "retina"